In [ ]:
# !pip install pyparsing==2.4.0
!pip install tensorflow==2.6.0
!pip install keras==2.6.0
import os
# os.chdir('/kaggle/working/w')
# import shutil
# shutil.rmtree("/kaggle/working/workspace")
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
sns.set_style('darkgrid')

from PIL import Image, ImageDraw
import tensorflow as tf


import ast
import sys
import time

import warnings
warnings.filterwarnings('ignore')

import greatbarrierreef


The below code will be useful in setting up TF object detection API. Part of it will be in the next notebook.Cheers! You can also refer to (https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html) for details.

In [ ]:
# data imports
DATA_PATH = '/kaggle/input/tensorflow-great-barrier-reef'
images_path = os.path.join(DATA_PATH,'train_images')
df_test = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/test.csv")
df_train = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/train.csv")
sample_submission = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/example_sample_submission.csv")
example = np.load("/kaggle/input/tensorflow-great-barrier-reef/example_test.npy")

In [ ]:
df_train['img_path'] = os.path.join('../input/tensorflow-great-barrier-reef/train_images')+"/video_"+df_train.video_id.astype(str)+"/"+df_train.video_frame.astype(str)+".jpg"
df_train['annotations'] = df_train['annotations'].apply(lambda x: ast.literal_eval(x))
df_train['Number_bbox'] = df_train['annotations'].apply(lambda x:len(x)) 

In [ ]:
def bbox_areas(annotations):
    if not annotations:
        return [0]
    area_list = []
    for annotation in annotations:
        area_list.append(annotation['width']*annotation['height'])
    return area_list
df_train["bbox_area"] = df_train["annotations"].apply(bbox_areas)
df_train["max_area"] = df_train["bbox_area"].apply(lambda x : max(x))
df_train["min_area"] = df_train["bbox_area"].apply(lambda x : min(x))



In [ ]:
def img_viz(df_train, id):
    image = df_train['img_path'][id]
    img = Image.open(image)
    
    for box in df_train['annotations'][id]:
        shape = [box['x'], box['y'], box['x']+box['width'], box['y']+box['height']]
        ImageDraw.Draw(img).rectangle(shape, outline ="red", width=3)
    display(img)
df_train.sort_values("max_area", ascending=False).head()


In [ ]:
# data imports
DATA_PATH = '/kaggle/input/tensorflow-great-barrier-reef'
images_path = os.path.join(DATA_PATH,'train_images')
df_test = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/test.csv")
df_train = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/train.csv")
sample_submission = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/example_sample_submission.csv")
example = np.load("/kaggle/input/tensorflow-great-barrier-reef/example_test.npy")
data = pd.read_csv("/kaggle/input/datagreatbarrier/data.csv") #processed df

In [ ]:
data

In [ ]:
actual_train_data = data.query("Number_bbox>0")

In [ ]:
!git clone https://github.com/tensorflow/models.git
# !cd models/research
# !export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
# !protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!wget -O protobuf.zip https://github.com/protocolbuffers/protobuf/releases/download/v3.19.0/protoc-3.19.0-linux-x86_64.zip -q
!unzip -o protobuf.zip
!rm protobuf.zip

In [ ]:
!pwd
!cd models/research
!pwd

In [ ]:
# %bash cd models/research
os.chdir('models/research')
# !pwd
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
import os

os.environ['AUTOGRAPH_VERBOSITY'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONPATH']=os.environ['PYTHONPATH']+':/kaggle/models/research/slim:/kaggle/models/research'
os.environ['PYTHONPATH']



In [ ]:
!cp object_detection/packages/tf2/setup.py .
!python -m pip install --use-feature=2020-resolver .

In [ ]:
!pwd
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
data['annotations'] = data['annotations'].apply(eval)


In [ ]:
os.chdir("/kaggle/working")
!mkdir workspace workspace/train_images workspace/test_images

In [ ]:
! mkdir /kaggle/working/workspace/annotations
label_map = """item {
    id: 1
    name: 'starfish'
}"""
with open("/kaggle/working/workspace/annotations/label_map.pbtxt", "w") as label_file:
    label_file.write(label_map)
label_file.close()
    

In [ ]:
# !cp /kaggle/input/reef-labels/labelmap.pbtxt /kaggle/working/workspace/annotations

In [ ]:
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from PIL import ImageDraw
from PIL import Image
import pandas as pd
import numpy as np
import json
import copy
import os
import cv2
import ast

# functions


In [ ]:
image_annotation_dict = dict(zip(data.img_path, data.annotations))
annotated_images = {key:value for key, value in image_annotation_dict.items() if len(value)>0}

In [ ]:
def get_xml_template(image_path, folder="/kaggle/working/workspace/train_images"):
    img = Image.open(image_path)
    img_width, img_height = img.size
    file_name = '_'.join(image_path.split('/')[-2:])
    image_info_dict = {
    "folder": folder.split('/')[-1],
    "filename": file_name,
    "path": folder + '/' + file_name,
    "source":{"database": "Unknown"},
    "size":{"width": str(img_width), "height": str(img_height), "depth":"3"},
    "segmented": "0",
        }
    return image_info_dict

def get_annotation_template(annotation):
    annotation_dict = {"name": "starfish",
                      "pose": "unknown",
                      "difficult": "0",
                      "bndbox":{"xmin": str(annotation['x']),
                              "xmax": str(annotation['x']+annotation['width']),
                              "ymin": str(annotation['y']),
                               "ymax": str(annotation['y']+annotation['height'])}}
    return annotation_dict

In [ ]:
import xml.etree.ElementTree as ET

def add_child(key, value, parent):
    child = ET.SubElement(parent, key)
    if isinstance(value, dict):
        for key_child, value_child in list(value.items()):
            add_child(key_child, value_child, child)
    else:
        child.text = value
    return  

def get_xml(image_data,folder="/kaggle/working/workspace/train_images"):
    image_info = get_xml_template(image_data[0], folder=folder)
    file_path = folder+'/'+image_info["filename"]
    xml_file_path = file_path.split('.')[0] +'.xml'
    annotation_list = image_data[1]
    root = ET.Element("annotations")
    for k,v in image_info.items():
        add_child(k,v,root)
    for annotation in annotation_list:
        annotation_root = ET.SubElement(root, "object")
        annotation_info = get_annotation_template(annotation)
        for k,v in annotation_info.items():
            add_child(k,v,annotation_root)
    return root,file_path,xml_file_path



In [ ]:
data.columns

In [ ]:
actual_train_data = data[data['Number_bbox']>0]

In [ ]:
from sklearn.model_selection import train_test_split
import shutil
train_data, test_data = train_test_split(actual_train_data,random_state=22, test_size=0.15)

In [ ]:
def transfer_images_xml(image_data_list, folder="/kaggle/working/workspace/train_images"):
    for image_data in tqdm(image_data_list):
        xml_details, image_path, xml_path = get_xml(image_data, folder=folder)
        shutil.copy(image_data[0], image_path)
        xml_tree = ET.ElementTree(xml_details)
        xml_tree.write(xml_path)

In [ ]:
test_image_annotation_dict = tuple(zip(test_data.img_path, test_data.annotations))


In [ ]:
transfer_images_xml(test_image_annotation_dict, folder="/kaggle/working/workspace/test_images")

Command to remove moved images

In [ ]:
# !rm /kaggle/working/workspace/test_images/*
# !rm /kaggle/working/workspace/train_images/*



In [ ]:
!mkdir /kaggle/working/workspace/scripts
!wget https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py -P /kaggle/working/workspace/scripts

In [ ]:
!python /kaggle/working/workspace/scripts/generate_tfrecord.py -x /kaggle/working/workspace/test_images -l /kaggle/working/workspace/annotations/label_map.pbtxt -o /kaggle/working/workspace/annotations/test.record

In [ ]:
train_image_annotation_dict = tuple(zip(train_data.img_path, train_data.annotations))
transfer_images_xml(train_image_annotation_dict, folder="/kaggle/working/workspace/train_images")


In [ ]:
!python /kaggle/working/workspace/scripts/generate_tfrecord.py -x /kaggle/working/workspace/train_images -l /kaggle/working/workspace/annotations/label_map.pbtxt -o /kaggle/working/workspace/annotations/train.record

In [ ]:
!wget download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz -P /kaggle/working/workspace/pre-trained-models/
!tar -xvzf /kaggle/working/workspace/pre-trained-models/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz


In [ ]:
config = """# Faster R-CNN with Resnet-50 (v1)
# Trained on COCO, initialized from Imagenet classification checkpoint

# Achieves -- mAP on COCO14 minival dataset.

# This config is TPU compatible.

model {
  faster_rcnn {
    num_classes: 1
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 640
        max_dimension: 640
        pad_to_max_dimension: true
      }
    }
    feature_extractor {
      type: 'faster_rcnn_resnet50_keras'
      batch_norm_trainable: true
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 14
    maxpool_kernel_size: 2
    maxpool_stride: 2
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        use_dropout: false
        dropout_keep_probability: 1.0
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
        share_box_across_classes: true
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.0
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 300
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
    use_static_shapes: true
    use_matmul_crop_and_resize: true
    clip_anchors_to_image: true
    use_static_balanced_label_sampler: true
    use_matmul_gather_in_matcher: true
  }
}

train_config: {
  batch_size: 64
  sync_replicas: true
  startup_delay_steps: 0
  replicas_to_aggregate: 8
  num_steps: 25000
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        cosine_decay_learning_rate {
          learning_rate_base: .04
          total_steps: 25000
          warmup_learning_rate: .013333
          warmup_steps: 2000
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  fine_tune_checkpoint_version: V2
  fine_tune_checkpoint: "/kaggle/working/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0"
  fine_tune_checkpoint_type: "classification"
  data_augmentation_options {
    random_horizontal_flip {
    }
  }

  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
  use_bfloat16: true  # works only on TPUs
}

train_input_reader: {
  label_map_path: "/kaggle/working/workspace/annotations/label_map.pbtxt"
  tf_record_input_reader {
    input_path: "/kaggle/working/workspace/annotations/train.record"
  }
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
  batch_size: 1;
}

eval_input_reader: {
  label_map_path: "/kaggle/working/workspace/annotations/label_map.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "/kaggle/working/workspace/annotations/test.record"
  }
}
"""

In [ ]:
file = open("/kaggle/working/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/pipeline.config", 'w')
file.truncate()
file.write(config)
file.close()

In [ ]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/model_main_tf2.py -P /kaggle/working/workspace/scripts

In [ ]:
# !pip install tensorflow-estimator==2.6.0
# !pip install tensorflow==2.6.0
# !pip install keras==2.6.0

In [ ]:
# !cd /kaggle/working/workspace/scripts

!python /kaggle/working/workspace/scripts/model_main_tf2.py  --logtostderr --model_dir=/kaggle/working/training --pipeline_config_path=/kaggle/working/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/pipeline.config